<a href="https://colab.research.google.com/github/sgrigerc/BasePython/blob/master/%D0%90%D0%B3%D1%80%D0%B5%D0%B3%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [23]:
#Загружаем датасеты
user_table = pd.read_csv('user_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
home_page = pd.read_csv('home_page_table.csv')



In [39]:
# Меняем название одинаковой колонки в каждом датасете
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation_page'})


In [40]:
# Склеиваем все датасеты все сразу через синтаксический сахар
df = user_table.merge(home_page, on = 'user_id', how = 'left')\
.merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left')\
.merge(payment_confirmation, on = 'user_id', how = 'left')

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [36]:
df['device'].unique()

array(['Desktop', 'Mobile'], dtype=object)

In [41]:
#Перекодируем названия событий в 0/1

df['payment_confirmation_page'] = df['payment_confirmation_page'].fillna(0)
condition = df['payment_confirmation_page'] == 'payment_confirmation_page'
df.loc[condition, 'payment_confirmation_page'] = 1
df.head()

df['payment_page'] = df['payment_page'].fillna(0)
condition = df['payment_page'] == 'payment_page'
df.loc[condition, 'payment_page'] = 1
df.head()

df['search_page'] = df['search_page'].fillna(0)
condition = df['search_page'] == 'search_page'
df.loc[condition, 'search_page'] = 1
df.head()

df['home_page'] = df['home_page'].fillna(0)
condition = df['home_page'] == 'home_page'
df.loc[condition, 'home_page'] = 1
df.head()


,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation_page
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [48]:
pd.pivot_table(df, index = 'sex', columns = 'device', values = 'home_page', aggfunc = 'sum', margins=True)


device,Desktop,Mobile,All
sex,,,
Female,29997,15078,45075
Male,30203,15122,45325
All,60200,30200,90400


In [59]:
#П1-построение сегментированной воронки конверсии пользователей интернет-магазина по полу

# Импортируем графическую библиотеку Plotly
from plotly import graph_objects as go

#Используем функцию crosstab
# (по умолчанию рассчитывает частоты, поэтому valus = df['user_id'] и aggfunc = 'count' можно не указывать)
df.groupby(['sex'])['user_id'].count()

dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation_page']].sum().unstack('sex').unstack('sex').reset_index()

# Уберем ненужный заголовок верхнего уровня
dfg.columns.name = None

# Переименуем стандартные после процедуры заголовки
dfg = dfg.rename(columns = {'index':'Page'})

# dfg.head()

# Настраиваем скрипт библиотеки Plotly нашими параметрами
fig = go.Figure()

# # Часть графика для типа полу
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

# Часть графика для типа устройств Mobile
fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()



In [60]:
user_table = pd.read_csv('user_table.csv')
user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [103]:
#П2- извлечение месяц и день недели
df = pd.read_csv('user_table.csv')

import datetime

#месяц
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
df['month'] = df['date'].dt.strftime('%m')

#день недели
df['day_of_week'] = df['date'].dt.weekday

df['DayWeek'] = df['date']. dt.day_name ()

df.head()

,user_id,date,device,sex,month,day_of_week,name_day
0,450007,2015-02-28,Desktop,Female,02,5,Saturday
1,756838,2015-01-13,Desktop,Male,01,1,Tuesday
2,568983,2015-04-09,Desktop,Male,04,3,Thursday
3,190794,2015-02-18,Desktop,Female,02,2,Wednesday
4,537909,2015-01-15,Desktop,Male,01,3,Thursday


In [132]:
#П3 - определить самый топовый день по продажам месяц и день недели sum или count

df = pd.read_csv('user_table.csv')\
.merge(payment_confirmation, on = 'user_id', how = 'left')

df['payment_confirmation_page'] = df['payment_confirmation_page'].fillna(0)
condition = df['payment_confirmation_page'] == 'payment_confirmation_page'
df.loc[condition, 'payment_confirmation_page'] = 1

import datetime

#месяц
df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
df['month'] = df['date'].dt.strftime('%m')

#день недели
df['day'] = df['date'].dt.weekday

df['DayWeek'] = df['date']. dt.day_name ()

# Агрегируем данные: подсчитываем количество продаж на каждый день недели
# summary = df[['day', 'payment_confirmation_page']].sum().reset_index()
# summary.head()

df.groupby(df.columns.tolist(),as_index=False).size()


,user_id,date,device,sex,payment_confirmation_page,month,day,DayWeek,size
0,17,2015-04-21,Desktop,Male,0,04,1,Tuesday,1
1,28,2015-04-29,Desktop,Male,0,04,2,Wednesday,1
2,37,2015-02-21,Mobile,Male,0,02,5,Saturday,1
3,38,2015-03-23,Mobile,Female,0,03,0,Monday,1
4,55,2015-02-01,Desktop,Male,0,02,6,Sunday,1
...,...,...,...,...,...,...,...,...,...
90395,999934,2015-04-16,Mobile,Female,0,04,3,Thursday,1
90396,999948,2015-02-26,Desktop,Female,0,02,3,Thursday,1
90397,999957,2015-04-30,Mobile,Female,0,04,3,Thursday,1
90398,999978,2015-04-14,Desktop,Female,0,04,1,Tuesday,1
